In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp

In [4]:
adata = sc.read('/home/julia/data/retina/data/27_07_qc_rpe_mel_for scienic.h5ad')

In [5]:
adata 

AnnData object with n_obs × n_vars = 19140 × 19737
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [6]:
adata.obs

,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt
AAACCTGAGCAAATCA-1,2627,2627,6767.0,226.0,3.339737
AAACCTGAGCGTTTAC-1,597,597,939.0,54.0,5.750799
AAACCTGAGCTAAACA-1,2091,2091,5172.0,180.0,3.480278
AAACCTGAGCTCCTCT-1,1953,1953,4953.0,122.0,2.463154
AAACCTGAGCTTTGGT-1,3203,3203,9837.0,865.0,8.793331
...,...,...,...,...,...
TTTGTCATCCACGTTC-1,864,864,1662.0,56.0,3.369435
TTTGTCATCGCGCCAA-1,3479,3479,10898.0,37.0,0.339512
TTTGTCATCGGAAACG-1,2123,2123,5560.0,49.0,0.881295
TTTGTCATCTTGACGA-1,1746,1746,4152.0,185.0,4.455684


In [7]:
# path to unfiltered loom file (this will be created in the optional steps below)
#f_loom_path_unfilt = "placods_unfiltered.loom" # test dataset, n=500 cells

# # path to loom file with basic filtering applied (this will be created in the "initial filtering" step below). Optional.
f_loom_path_scenic = "RPE_mel_filtered_scenic.loom"

# path to anndata object, which will be updated to store Scanpy results as they are generated below
f_anndata_path = "anndata.h5ad"

# path to pyscenic output
f_pyscenic_output = "RPE_mel_pyscenic_output.loom"

# loom output, generated from a combination of Scanpy and pySCENIC results:
f_final_loom = 'RPE_mel_scenic_integrated-output.loom'

In [8]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)

In [23]:
f_loom_path_scenic

'RPE_mel_filtered_scenic.loom'

In [7]:
f_tfs = '/home/julia/data/Scenic_database/Mouse/TF/allTFs_mm.txt'

In [11]:
!/home/julia/miniconda3/envs/scenicplus/bin/pyscenic grn /home/julia/data/retina/notebook/Scienic/RPE_mel_filtered_scenic.loom /home/julia/data/Scenic_database/Mouse/TF/allTFs_mm.txt -o adj_1.csv  --num_workers 12


2024-06-27 11:51:20,609 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-06-27 11:51:25,346 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
2024-06-27 11:51:33,247 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-szr6iud8', purging
2024-06-27 11:51:33,247 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-m7ruz91g', purging
2024-06-27 11:51:33,248 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-zlqllkw2', purging
2024-06-27 11:51:33,248 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-46xw0fmx', purging
2024-06-27 11:51:33,248 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-0yqukhpk', purging
2024-06-27 11:51:33,248 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-

!pyscenic grn {f_loom_path_scenic} {f_tfs} -o adj.csv --num_workers 10

In [12]:
adjacencies = pd.read_csv("adj_1.csv", index_col=False)

In [13]:
adjacencies.head()

,TF,target,importance
0,Mylk,Myl9,381.207172
1,Mylk,Acta2,359.941265
2,Mylk,Tpm2,342.049943
3,Mylk,Sparcl1,339.948033
4,Mylk,Mustn1,298.719392


In [ ]:
# path to unfiltered loom file (this will be created in the optional steps below)
#f_loom_path_unfilt = "placods_unfiltered.loom" # test dataset, n=500 cells

# # path to loom file with basic filtering applied (this will be created in the "initial filtering" step below). Optional.
f_loom_path_scenic = "RPE_mel_filtered_scenic.loom"

# path to anndata object, which will be updated to store Scanpy results as they are generated below
f_anndata_path = "anndata.h5ad"

# path to pyscenic output
f_pyscenic_output = "RPE_mel_pyscenic_output.loom"

# loom output, generated from a combination of Scanpy and pySCENIC results:
f_final_loom = 'RPE_mel_scenic_integrated-output.loom'

In [21]:
import glob
f_db_glob = "/home/julia/data/Scenic_database/Mouse/Gene_based/*feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

In [22]:
f_db_names

'/home/julia/data/Scenic_database/Mouse/Gene_based/mm10_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather /home/julia/data/Scenic_database/Mouse/Gene_based/mm10_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather'

In [32]:
!/home/julia/miniconda3/envs/scenicplus/bin/pyscenic ctx adj_1.csv \
    {f_db_names} \
    --annotations_fname /home/julia/data/Scenic_database/Mouse/Motif2TF/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl \
    --expression_mtx_fname /home/julia/data/retina/notebook/Scienic/RPE_mel_filtered_scenic.loom \
    --output reg_01_07.csv \
    --mask_dropouts \
    --num_workers 20


2024-07-01 11:07:20,792 - pyscenic.cli.pyscenic - INFO - Creating modules.

2024-07-01 11:07:23,120 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-07-01 11:07:29,098 - pyscenic.utils - INFO - Calculating Pearson correlations.

2024-07-01 11:07:30,379 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2024-07-01 11:08:43,768 - pyscenic.utils - INFO - Creating modules.

2024-07-01 11:11:11,020 - pyscenic.cli.pyscenic - IN

In [36]:
!/home/julia/miniconda3/envs/scenicplus/bin/pyscenic aucell \
    /home/julia/data/retina/notebook/Scienic/RPE_mel_filtered_scenic.loom \
    /home/julia/data/retina/notebook/Scienic/reg_01_07.csv \
    --output /home/julia/data/retina/notebook/Scienic/RPE_mel_pyscenic_output.loom \
    --num_workers 20


2024-07-01 11:38:23,243 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-07-01 11:38:27,991 - pyscenic.cli.pyscenic - INFO - Loading gene signatures.
Create regulons from a dataframe of enriched features.
Additional columns saved: []

2024-07-01 11:38:32,293 - pyscenic.cli.pyscenic - INFO - Calculating cellular enrichment.

2024-07-01 11:39:18,844 - pyscenic.cli.pyscenic - INFO - Writing results to file.
/home/julia/miniconda3/envs/scenicplus/lib/python3.8/site-packages/pyscenic/cli/utils.py:347: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, threshold in auc_thresholds.iteritems()
